In [5]:
# 단순히 눈꺼풀 위, 아래의 y좌표를 직접 추출하여 눈꺼풀 거리가 일정 이하면 감긴것으로 추정
# mediapipe는 사람 얼굴이 인식되어야 눈의 좌표를 찾기 때문에 음영값을 이용하여 눈꺼풀의 위치 추정
# 임계값을 조정하며 최적의 임계값을 찾을 수 있음
# test - closed 1666개
# test - open 5325개
# train - closed 33322개
# train - open 106482개
# val - closed 6665 개
# val - open 21296 개
# 최소 1666개 이므로 이에 맞춰서 하는 것을 추천
# 대략 55%대를 유지하는 정확도를 가짐

import os
import cv2
import numpy as np
import random

MAX_PER_CLASS = 1666

# -------------------------
# [1] 눈 이미지에서 눈꺼풀 위·아래 y 좌표 직접 추출
# -------------------------
def extract_eye_top_bottom(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 눈동자는 보통 어두워서 역이진화 (어두운 영역을 흰색으로)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # 수평 픽셀 합 (y축 방향)
    horizontal_sum = np.sum(thresh, axis=1)

    ys = np.where(horizontal_sum > 0)[0]

    if len(ys) == 0:
        return None

    top = ys[0]
    bottom = ys[-1]

    return top, bottom

# -------------------------
# [2] 이미지 폴더 순회하며 데이터(눈꺼풀 거리, 라벨) 수집
# -------------------------
def load_eye_data(base_dir, max_per_class=MAX_PER_CLASS):
    data = []

    for label_name in ['open', 'closed']:
        folder = os.path.join(base_dir, label_name)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
        files = random.sample(files, min(max_per_class, len(files)))

        label = 1 if label_name == 'open' else 0

        for file in files:
            path = os.path.join(folder, file)
            img = cv2.imread(path)
            if img is None:
                continue

            coords = extract_eye_top_bottom(img)
            if coords:
                top, bottom = coords
                distance = bottom - top
                data.append((distance, label))

    return data

# -------------------------
# [3] 임계값 기반 눈 감김 분류기
# -------------------------
def simple_threshold_classifier(data, threshold):
    correct = 0
    total = 0

    for distance, label in data:
        pred = 0 if distance < threshold else 1
        if pred == label:
            correct += 1
        total += 1

    accuracy = correct / total * 100 if total > 0 else 0
    return accuracy

# -------------------------
# [4] 데이터 경로 설정 및 데이터 로딩
# -------------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir = r"C:\Users\Users\open-closed-eyes-dataset\test"

train_data = load_eye_data(train_dir)
val_data = load_eye_data(val_dir)
test_data = load_eye_data(test_dir)

print(f"학습 샘플 수: {len(train_data)} / 검증: {len(val_data)} / 테스트: {len(test_data)}")

# -------------------------
# [5] 경험적으로 정한 임계값 (픽셀 단위, 눈 이미지 크기에 따라 조정 필요)
# -------------------------
threshold = 5

# -------------------------
# [6] 정확도 평가
# -------------------------
train_acc = simple_threshold_classifier(train_data, threshold)
val_acc = simple_threshold_classifier(val_data, threshold)
test_acc = simple_threshold_classifier(test_data, threshold)

print(f"Train Accuracy: {train_acc:.2f}%")
print(f"Validation Accuracy: {val_acc:.2f}%")
print(f"Test Accuracy: {test_acc:.2f}%")

학습 샘플 수: 2769 / 검증: 2747 / 테스트: 2753
Train Accuracy: 55.04%
Validation Accuracy: 55.81%
Test Accuracy: 55.87%


In [7]:
# 최적의 threshold 를 찾기 위한 코드

import os
import cv2
import numpy as np
import random

MAX_PER_CLASS = 1666

# -------------------------
# [1] 눈 이미지에서 눈꺼풀 위·아래 y 좌표 직접 추출
# -------------------------
def extract_eye_top_bottom(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    horizontal_sum = np.sum(thresh, axis=1)
    ys = np.where(horizontal_sum > 0)[0]
    if len(ys) == 0:
        return None
    top = ys[0]
    bottom = ys[-1]
    return top, bottom

# -------------------------
# [2] 이미지 폴더 순회하며 데이터(눈꺼풀 거리, 라벨) 수집
# -------------------------
def load_eye_data(base_dir, max_per_class=MAX_PER_CLASS):
    data = []
    for label_name in ['open', 'closed']:
        folder = os.path.join(base_dir, label_name)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
        files = random.sample(files, min(max_per_class, len(files)))
        label = 1 if label_name == 'open' else 0
        for file in files:
            path = os.path.join(folder, file)
            img = cv2.imread(path)
            if img is None:
                continue
            coords = extract_eye_top_bottom(img)
            if coords:
                top, bottom = coords
                distance = bottom - top
                data.append((distance, label))
    return data

# -------------------------
# [3] 임계값 기반 눈 감김 분류기
# -------------------------
def simple_threshold_classifier(data, threshold):
    correct = 0
    total = 0
    for distance, label in data:
        pred = 0 if distance < threshold else 1
        if pred == label:
            correct += 1
        total += 1
    return correct / total * 100 if total > 0 else 0

# -------------------------
# [4] 경로 및 데이터 로딩
# -------------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir = r"C:\Users\Users\open-closed-eyes-dataset\test"

train_data = load_eye_data(train_dir)
val_data = load_eye_data(val_dir)
test_data = load_eye_data(test_dir)

print(f"📊 학습 샘플 수: {len(train_data)} / 검증: {len(val_data)} / 테스트: {len(test_data)}")
print()

# -------------------------
# [5] 다양한 threshold 값에 대한 정확도 출력
# -------------------------
print("📐 Threshold별 Accuracy (Train / Validation / Test)")
print("-------------------------------------------------------")
for threshold in range(1, 25):
    train_acc = simple_threshold_classifier(train_data, threshold)
    val_acc = simple_threshold_classifier(val_data, threshold)
    test_acc = simple_threshold_classifier(test_data, threshold)

    print(f"Threshold = {threshold:2d} | Train: {train_acc:6.2f}% | Val: {val_acc:6.2f}% | Test: {test_acc:6.2f}%")

📊 학습 샘플 수: 2740 / 검증: 2766 / 테스트: 2740

📐 Threshold별 Accuracy (Train / Validation / Test)
-------------------------------------------------------
Threshold =  1 | Train:  54.78% | Val:  55.64% | Test:  54.85%
Threshold =  2 | Train:  55.29% | Val:  55.82% | Test:  55.04%
Threshold =  3 | Train:  55.33% | Val:  55.89% | Test:  55.33%
Threshold =  4 | Train:  55.26% | Val:  55.57% | Test:  55.15%
Threshold =  5 | Train:  55.33% | Val:  55.53% | Test:  55.33%
Threshold =  6 | Train:  55.44% | Val:  55.46% | Test:  55.58%
Threshold =  7 | Train:  55.33% | Val:  55.57% | Test:  55.55%
Threshold =  8 | Train:  55.18% | Val:  55.42% | Test:  55.58%
Threshold =  9 | Train:  55.15% | Val:  55.39% | Test:  55.55%
Threshold = 10 | Train:  55.22% | Val:  55.17% | Test:  55.62%
Threshold = 11 | Train:  55.15% | Val:  54.95% | Test:  55.58%
Threshold = 12 | Train:  55.00% | Val:  54.59% | Test:  55.58%
Threshold = 13 | Train:  54.78% | Val:  54.37% | Test:  55.11%
Threshold = 14 | Train:  54.45% | V